In [2]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
import os
from collections import Counter

In [8]:
def packetReceived(parsed):
    return "name" in parsed and parsed["name"] == "transport:packet_received"

def connectionClose(parsed):
    return "name" in parsed and parsed["name"] == "transport:connection_closed"

def packetSent(parsed):
    return "name" in parsed and parsed["name"] == "transport:packet_sent"

def is1RTTPacket(parsed):
    return (packetReceived(parsed) or packetSent(parsed)) and parsed["data"]["header"]["packet_type"] == "1RTT"

def isHandshakePacket(parsed):
    return (packetReceived(parsed) or packetSent(parsed)) and parsed["data"]["header"]["packet_type"] == "handshake"
    
def printPacketInformation(parsed):
    isSending = packetSent(parsed)
    isReceived = packetReceived(parsed)
    if not isSending and not isReceived:
        return
    header = parsed["data"]["header"]
    
    message = str(parsed["time"]) + " "
    if isSending:
        message += "Sending "
    elif isReceived:
        message += "Received " 
        
    message += header["packet_type"] + " "
    message += str(header["packet_number"]) if "packet_number" in header else ""
    print(message)

def isRetryPacket(parsed):
    return (packetReceived(parsed) 
        and "data" in parsed 
        and "header" in parsed["data"] 
        and parsed["data"]["header"]["packet_type"] == "retry"
        and "token" in parsed["data"]["header"]
        and parsed["data"]["header"]["token"])

def isVersionNegotiation(parsed):
    return (packetReceived(parsed) 
        and "data" in parsed 
        and "header" in parsed["data"] 
        and parsed["data"]["header"]["packet_type"] == "version_negotiation")

def isIdleTimeout(parsed):
    return (connectionClose(parsed) and "trigger" in parsed["data"] and parsed["data"]["trigger"] == "idle_timeout")

def getRetryToken(parsed):
    return parsed["data"]["header"]["token"]["data"]

def isPTOExpire(parsed):
    return "name" in parsed and parsed["name"] == "recovery:loss_timer_updated" and parsed["data"]["event_type"] == "expired"

def isInitialPacket(parsed):
    return ((packetSent(parsed) or packetReceived(parsed)) and parsed["data"]["header"]["packet_type"] == "initial")

def isPacketDropped(parsed):
    return ("name" in parsed and parsed["name"] == "transport:packet_dropped")

def isMetricsUpdated(parsed):
    return ("name" in parsed and parsed["name"] == "recovery:metrics_updated")

def getRTTValues(parsed):
    if not isMetricsUpdated(parsed):
        return None
    data = parsed["data"]
    result = {}
    if "latest_rtt" in data:
        result["latest_rtt"] = data["latest_rtt"]
    if "rtt_variance" in data:
        result["rtt_variance"] = data["rtt_variance"]
    return result

def containsHandshakeDoneFrame(parsed):
    return (packetReceived(parsed) and "data" in parsed and "frames" in parsed["data"] and any(frame["frame_type"] == "handshake_done" for frame in parsed["data"]["frames"]))

def getConnectionStats(filename):
    with open(filename, "r") as f:
        data = { "success": True, "retry": False, "version_negotiation": False }
        for line in f:
            try:
                parsed = json.loads(line)
            except:
                return { "success": False, "retry": False, "version_negotiation": False }
            if isRetryPacket(parsed):
                data["retry"] = True
            elif isVersionNegotiation(parsed):
                print("negotiated")
                data["version_negotiation"] = True
            elif isIdleTimeout(parsed):
                data["success"] = False
        return data

In [3]:
connections = {}

for filename in os.listdir("qlog-results"):
    ip, port, _ = filename.split("-")
    stats = getConnectionStats("qlog-results/" + filename)
    if ip not in connections:
        connections[ip] = {}
        connections[ip][port] = stats
    else:
        connections[ip][port] = stats

with open("qlog-results.json", "w+") as f:
    f.write(json.dumps(connections, indent = 4))

### Detailed Analysis

In [37]:
filename = "94.140.14.14-784-fd1c303e463e6b52.qlog"
# filename = "139.162.70.69-784-96d721d04d5b8c4e6a.qlog"

In [38]:
def printRecords(records):
    counter = Counter()
    variances = []
    rtt_values = []
    time_sent = {}
    time_received = {}
    
    def testCoalescedSent(record):
        if record["time"] not in time_sent:
                time_sent[record["time"]] = True
                counter["coalesced_sent"] += 1
    
    def testCoalescedReceived(record):
        if record["time"] not in time_received:
                time_received[record["time"]] = True
                counter["coalesced_received"] += 1
    
    for record in records:
        done = False
        if isPTOExpire(record):
            counter["pto_expired"] += 1
        if isIdleTimeout(record):
            return None
            
        if done:
            continue
        if isInitialPacket(record) and packetSent(record):
            counter["initial_sent"] += 1
            testCoalescedSent(record)
            if "retry_received" in counter:
                counter["initial_sent_after_retry"] += 1
            else:
                counter["initial_sent_until_retry"] += 1
        if isInitialPacket(record) and packetReceived(record):
            testCoalescedReceived(record)
            counter["initial_received"] += 1
        if isHandshakePacket(record) and packetSent(record):
            testCoalescedSent(record)
            counter["handshake_sent"] += 1
        if isHandshakePacket(record) and packetReceived(record):
            testCoalescedReceived(record)
            counter["handshake_received"] += 1
        if isRetryPacket(record):
            testCoalescedReceived(record)
            counter["retry_received"] += 1
        if isPacketDropped(record):
            counter["packet_dropped"] += 1
        if isMetricsUpdated(record):
            values = getRTTValues(record)
            if "latest_rtt" in values:
                rtt_values.append(values["latest_rtt"])
            if "rtt_variance" in values:
                variances.append(values["rtt_variance"])
        if containsHandshakeDoneFrame(record):
            done = True
            counter["total_sent"] += counter["handshake_sent"] + counter["initial_sent"]
            counter["total_received"] += counter["inital_received"] + counter["handshake_received"] + counter["retry_received"]
    
    df = pd.DataFrame({ "rtt": rtt_values})
    rtt = df["rtt"].median()
    
    df = pd.DataFrame({ "variance": variances})
    var = df["variance"].median()
    
    return { **dict(counter), "rtt": rtt, "var": var }
        
def analyzeSpecificFile(filename):
    records = []

    with open("qlog-results/" + filename, "r") as f:
        for line in f:
            records.append(json.loads(line))
    
    return printRecords(records)

analyzeSpecificFile(filename)

{'initial_sent': 3,
 'coalesced_sent': 4,
 'initial_sent_until_retry': 1,
 'coalesced_received': 4,
 'retry_received': 1,
 'initial_sent_after_retry': 2,
 'initial_received': 1,
 'handshake_received': 4,
 'handshake_sent': 2,
 'total_sent': 5,
 'total_received': 5,
 'rtt': 11.1045,
 'var': 4.567}

In [39]:
folder = "qlog-results"

allResult = []

for filename in os.listdir("qlog-results"):
    ip, port, _ = filename.split("-")
    records = []
    with open(folder + "/" + filename, "r") as f:
        for line in f:
            records.append(json.loads(line))
    statsResult = printRecords(records)
    if statsResult == None:
        continue
    result = dict(statsResult)
    result["ip"] = ip
    result["port"] = port
    allResult.append(result)

In [40]:
test = pd.DataFrame(allResult)
test = test.set_index("ip")
test.to_csv("qlog-stats.csv")